In [1]:
import os
import pandas as pd
import nltk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from nltk.tokenize import TweetTokenizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
nltk.download('stopwords')
#import calendar
nltk.download('averaged_perceptron_tagger')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chuckclarke/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/chuckclarke/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
import tweepy as tw
import re
import networkx
import matplotlib.pyplot as plt
from wordcloud import WordCloud, ImageColorGenerator
#from PIL import Image
import numpy as np
import tkinter as tk
from tkinter.ttk import Progressbar

In [3]:
# Clean text

def remove_url(txt):
    raw = BeautifulSoup(txt).get_text()
    
    
    cleaned = re.sub(r"@+", " ", raw)
    #cleaned lower case
  
    cleaned = cleaned.lower()
    cleaned =re.sub('@[a-z]+', ' ', cleaned)
    cleaned = re.sub('t.co+', ' ', cleaned)
    cleaned = re.sub('/', ' ', cleaned)
    cleaned = re.sub('[^A-Za-z]+', ' ', cleaned)

    # First we remove inline JavaScript/CSS:
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", "", cleaned.strip())
    # Then we remove html comments. This has to be done before removing regular
    # tags since comments can contain '>' characters.
    cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", "", cleaned)
    # Next we can remove the remaining tags:
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
    # Finally, we deal with whitespace
    cleaned = re.sub(r"&nbsp;", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    
    # remove https  
    cleaned = re.sub(r"https", " ", cleaned)
    #
    #cleaned = re.sub(r"@+", " ", cleaned)
    #cleaned= re.sub('[^a-zA-Z0-9]', '', cleaned) 
    tknzr = TweetTokenizer(r'\w+')
    tokens = tknzr.tokenize(cleaned)
   
    return  tokens


In [8]:
class Application(object):
   
    #declare variables 
    
    txt_box = tk.Text()
    e1 = tk.Entry()
    e2 = tk.Entry()
    val_list =[]
    df = pd.DataFrame ( columns = ['word','word_freq'])
    ax =""
    # Class initial startup
    def __init__(self,window):
        self.window = window
        super(Application,self).__init__()
         
         # call the below function to create the GUI window   
        self.create_gui_window()
    
    #pass search parameter 
    def pass_parameters(self,api):
         self.bar.step(10)
         search_words = self.e1.get()
         new_search  = search_words + " -filter:retweets"   
         date_since = self.e2.get()
        
         # call twitter api
         tweets = tw.Cursor(api.search, wait_on_rate_limit=True,
              q=new_search,
              lang="en",
              since=date_since).items(500)
            
          #Clear entry textbox  
         self.e1.delete(0, tk.END)
         self.e2.delete(0, tk.END)
            
         self.bar.step(30)
        
         return tweets  
        
        
      # API keys and secret info  
    def setup_api_access(self):

        twitter_keys = {
        'consumer_key':        'kiv7nzQLajBWoS4X5EyOrAsK8',
        'consumer_secret':     'EBhwHe7TpzEiRa3P4IvJ5OKs49DeRhQMH3w7308HqanQ4EYBRE',
        'access_token_key':    '1187719283180232707-DCiFFmgm5yE4xVUo1TjYXvNrEdvGo1',
        'access_token_secret': 'ZNwuYmIBUzwh00WPioBC5A3vy6DuyR4PdWJqAx46kWhzJ'
         }

         #Setup access to API
        auth = tw.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
        auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

        return tw.API(auth)
    
    # Request API data/ check for value in parameter
    
    def start_program_run(self):
        
        self.bar.step(0)
        self.clear_textbox()
        self.bar.step(20)
        
        if len(self.e1.get())==0:
            tk.messagebox.showwarning("Warning","You are missing a search parameter")
            self.bar.step(0)
        else:    
            api = self.setup_api_access()
            self.list_of_tweets =[]
            self.list_of_tweets = [(tweet.text) for tweet in self.pass_parameters(api)]
            self.bar.step(50)
            self.update_textbox(self.list_of_tweets)
            self.bar.step(100)
        
    # Main GUI  interface code 
    
    def create_gui_window(self):
    
        sizex = 1400
        sizey = 700
        posx  = 40
        posy  = 20
   

        #self.window = tk.Tk()
        self.window.wm_geometry("%dx%d+%d+%d" % (sizex, sizey, posx, posy))
        self.window.config(background = "blue") 
        
        #Setup label GUI
        tk.Label(self.window,bg='green' ,
         text="Enter search word").place(x=30, y=25, height=20, width=150)
        tk.Label(self.window, bg='green',
          text="Enter Date").place(x=30, y=50, height=20, width=150)
        
        # progress bar
        self.bar = Progressbar(self.window,orient="horizontal",length=300,mode="determinate")
        #bar['value'] = 80
        self.bar.place(x=730, y=25, height=30, width=450)
        
        tk.Label(self.window,bg='yellow' ,
         text="Progress Bar").place(x=730, y=15, height=20, width=150)
        
        
        #Setup  Entry GUI
        self.e1 = tk.Entry(self.window , bg ='peachpuff')
        self.e2 = tk.Entry(self.window,bg = 'peachpuff')
    
        self.e1.place(x=185, y=25, height=20, width=150)
        self.e2.place(x=185, y=50, height=20, width=150)
        # setUP  buttons GUI
        
        self.btn_run = tk.Button(self.window, 
            text='Run Program', bg='yellow',highlightbackground='#3E4149', 
                  command=self.start_program_run, state=tk.NORMAL).place(x=70, y=110, height=20, width=150)
        
        self.btn_clean = tk.Button(self.window, 
                                   bg='yellow', highlightbackground='#3E4149',text='Process and clean text',
                  command = self.process_text).place(x=230, y=110, height=20, width=150)
        
        
        self.btn_quit =tk.Button(self.window, 
                                   bg='yellow',highlightbackground='#3E4149', text='Quit Program', 
                  command=self.stop_program).place(x=390, y=110, height=20, width=150)
       
        
       
        self.scroll = tk.Scrollbar(self.window)
        self.scroll.config(command=self.txt_box.yview)
        self.txt_box.config(yscrollcommand=self.scroll.set)
       
        self.txt_box1 = tk.Text(self.window,foreground="white",  # Set the text color to white
            background="black"  # Set the background color to black
                               , height=25, width=275, xscrollcommand=self.scroll.set
                                ) 
    
    
        self.txt_box2 = tk.Text(self.window,foreground="white",  # Set the text color to white
            background="black"  # Set the background color to black
                               , height=20, width=75
                                ) 
        
        self.txt_box3 = tk.Text(self.window,foreground="white",  # Set the text color to white
            background="black"  # Set the background color to black
                               , height=205, width=75
                                ) 
        
        tk.Label(self.window,bg='yellow' ,
                                  text="Part Of Speech").place(x=60, y=475, height=20, width=150)
        
        tk.Label(self.window,bg='yellow' ,
                                   text="Twenty Top Words").place(x=360, y=475, height=20, width=150)
        
        self.txt_box1.place(x=60, y=150, height=300, width=550)
        
        self.scroll.place(x=510, y=150, height=300, width=10)
        
        self.txt_box2.place(x=60, y=500, height=200, width=250)
        
        self.txt_box3.place(x=360, y=500, height=200, width=250)
            
        #self.create_matplot_chart()
        
        # start main window GUI
        
        #self.window.mainloop()
      
   
    def  create_matplot_chart(self):
       
        self.figure = plt.Figure(figsize=(4,4), dpi=100)
        self.canvas = FigureCanvasTkAgg(self.figure, self.window)
        self.canvas.get_tk_widget().place(x=630, y=150, height=500, width=720)
     
        self.ax = self.figure.add_subplot(111)
        self.ax.set_xlabel('Word Frequency')
        self.ax.set_ylabel('Count')
        self.df.plot('word','word_freq',kind='line', legend=True, ax=self.ax, color='r',marker='o', fontsize=10)
       # canvas.get_tk_widget().place(x=560, y=150, height=300, width=450)
        
    def update_textbox(self , list_of_tweets):
            self.txt_box1.insert('1.0', list_of_tweets)
       
        
    def clear_textbox(self):
        
            self.figure = None
            self.txt_box1.delete('1.0', tk.END) # clear the entry field
            self.txt_box2.delete('1.0', tk.END) # clear the entry field
            self.txt_box3.delete('1.0', tk.END) # clear the entry field
       
    # process text by cleaning all HTML tags, spaces, special characters, and punctuations
    def process_text(self):
       
        self.bar.step(0)
        
        self.new_list= []
        new_str = ""
        for strr in self.list_of_tweets :
            self.new_list.append(remove_url(strr))
            
        self.bar.step(20)   
        # remove unnecessary stop words  
        self.remove_stop_word(self.new_list) 
        
        self.bar.step(100)
        
    def remove_stop_word(self, new_list) :
        
        val_list = []
        str_of_token = ""
        sr= stopwords.words('english')
        clean_tokens = new_list[:]
        token_string =""
        for words in self.new_list:
            var= [word for word in words if not word in stopwords.words()]
            self.val_list = self.val_list + var
        self.bar.step(50)   
        self.txt_box1.insert('1.0', self.val_list)
        
        self.bar.step(60)
        self.freq_dist(self.val_list)
         
    def freq_dist(self,val_list):
        self.freq = nltk.FreqDist( val_list)
        
        list_x =[]
        list_y = []
        for key,val in self.freq.items():
            list_x.append(str(key))
            list_y.append( int(val))
        self.df = pd.DataFrame ( columns = ['word','word_freq'])
        self.df["word"]= list_x
        self.df["word_freq"]= list_y
        
        self.bar.step(70)
        self.part_of_speech()
        
        self.bar.step(80)
        self.most_freq_twenty_words()
        
        self.bar.step(90)
        self.create_matplot_chart()
        self.bar.step(0)
        
        #self.word_cloud_image()
        
       
        
    def word_cloud(self):
        wc = WordCloud(max_font_size=90).generate(str(self.val_list))
        wc
    def word_cloud_image(self):
        
        custom_mask = np.array(Image.open("jean-philippe.jpg"))
        wc = WordCloud(background_color="white", mask=custom_mask,max_font_size =70)
        wc.generate(str(self.val_list))
        plt.imshow(wc, interpolation='bilinear')
        plt.axis("off")
        plt.show()
     
    def part_of_speech(self):
         try:  
            tagged = nltk.pos_tag(self.val_list)
            self.txt_box2.insert('1.0', tagged)
         except Exception as e:
              print(str(e))

    def most_freq_twenty_words(self):
        
        self.txt_box3.insert('1.0', self.freq.most_common(20))
        
    def stop_program(self):
          
        self.window.destroy() 
            

In [9]:
window = tk.Tk()

In [10]:


app = Application(window)

In [11]:
window.mainloop()

In [ ]:
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
custom_mask = np.array(Image.open("jean-philippe.jpg"))
wc = WordCloud(background_color="white", mask=custom_mask,max_font_size =70)
wc.generate(str(val_list))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()